In [9]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [10]:
df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
df = df.drop(['ID', 'ZIP Code'], axis=1)
df = pd.get_dummies(df, columns=['Education', 'Personal Loan', 'Securities Account', 'CD Account', 'Online', 'CreditCard'],drop_first=True)
x = df.drop('Personal Loan_1', axis=1)
y = df['Personal Loan_1']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [12]:
def build_model():
    model = Sequential([
        Dense(32, input_shape=(x_train.shape[1],), activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')])
    return model

In [13]:
def fitness_function(weights):
    model.set_weights(weights)
    y_pred = model.predict(x_train)
    accuracy = K.mean(K.equal(K.round(y_pred), y_train))
    return accuracy

In [14]:
def cultural_algorithm(population_size, max_generation):
    global model
    num_weights = len(model.get_weights())
    population = [model.get_weights() for i in range(population_size)]
    for generation in range(max_generation):
        fitness = np.array([fitness_function(individual) for individual in population])
        sorted_indices = np.argsort(fitness)[::-1]
        population = [population[i] for i in sorted_indices]
        cultural_best = population[0]
        for i in range(population_size):
            for j in range(num_weights):
                if np.random.rand() < 0.3:
                    population[i][j] = cultural_best[j]
        for i in range(2, population_size):
            parent1 = population[np.random.randint(2)]
            parent2 = population[np.random.randint(2)]
            child = parent1.copy()
            for j in range(num_weights):
                if np.random.rand() < 0.5:
                    child[j] = parent2[j]
            if np.random.rand() < 0.1:
                child[np.random.randint(num_weights)] += np.random.normal(scale=0.1)
            population[i] = child
    fitness = np.array([fitness_function(individual) for individual in population])
    sorted_indices = np.argsort(fitness)[::-1]
    population = [population[i] for i in sorted_indices]
    best_individual = population[0]
    return best_individual

In [17]:
model = build_model()
population_size = 10
max_generation = 10
population = [model.get_weights() for i in range(population_size)]

In [18]:
best_individual = cultural_algorithm(population_size, max_generation)
model.set_weights(best_individual)

118/118 [==============================] - 0s 1ms/step


In [28]:
y_pred = model.predict(x_test)
np.round(y_pred)

40/40 [==============================] - 0s 2ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)